In [ ]:
import pandas as pd
import numpy as np
from os import listdir
from os import path
import pytz
import glob
from dateutil.parser import isoparse
from datetime import datetime, timedelta
import math
import time
from DirectoryGenerator import DirectoryGenerator

In [ ]:
dirGen = DirectoryGenerator()

In [ ]:
startDate = datetime.strptime(str(input("Start Date (Format: YYYY-MM-DD): ")), "%Y-%m-%d").astimezone(pytz.utc)
startDate

In [ ]:
endDate = datetime.strptime(str(input("End Date (Format: YYYY-MM-DD): ")), "%Y-%m-%d").astimezone(pytz.utc)
endDate

In [ ]:
totalDays = abs(endDate - startDate).days + 1
totalDays

In [ ]:
def parseTimestamp(timestamp):
    try:
        return isoparse(timestamp).astimezone(pytz.utc)
    except:
        return np.nan

In [ ]:
assignmentsDataCommentsFilesPath = str(input())

In [ ]:
assignmentsDataCommentsFiles = glob.glob(assignmentsDataCommentsFilesPath + dirGen.getDelimiter() + '**' + dirGen.getDelimiter() + '*.csv', recursive=True)

In [ ]:
assignmentsDataCommentsData = pd.DataFrame()
for assignmentsDataCommentsFile in assignmentsDataCommentsFiles:
    assignmentsDataCommentsData = assignmentsDataCommentsData.append(pd.read_csv(assignmentsDataCommentsFile), ignore_index = True)
    
assignmentsDataCommentsData

In [ ]:
assignmentsDataCommentsData['metadata_event_time'] = assignmentsDataCommentsData['metadata_event_time'].apply(parseTimestamp)
assignmentsDataCommentsData.dropna(subset=['metadata_event_time'], inplace = True)
assignmentsDataCommentsData = assignmentsDataCommentsData.loc[(assignmentsDataCommentsData['metadata_event_time'] >= startDate) & (assignmentsDataCommentsData['metadata_event_time'] <= endDate)]
assignmentsDataCommentsData

In [ ]:
assignmentsDataSubmissionsFilesPath = str(input())

In [ ]:
assignmentsDataSubmissionsFiles = glob.glob(assignmentsDataSubmissionsFilesPath + dirGen.getDelimiter() + '**' + dirGen.getDelimiter() + '*.csv', recursive=True)

In [ ]:
assignmentsDataSubmissionsData = pd.DataFrame()
for assignmentsDataSubmissionsFile in assignmentsDataSubmissionsFiles:
    assignmentsDataSubmissionsData = assignmentsDataSubmissionsData.append(pd.read_csv(assignmentsDataSubmissionsFile), ignore_index = True)
    
assignmentsDataSubmissionsData

In [ ]:
assignmentsDataSubmissionsData['metadata_event_time'] = assignmentsDataSubmissionsData['metadata_event_time'].apply(parseTimestamp)
assignmentsDataSubmissionsData.dropna(subset=['metadata_event_time'], inplace = True)
assignmentsDataSubmissionsData = assignmentsDataSubmissionsData.loc[(assignmentsDataSubmissionsData['metadata_event_time'] >= startDate) & (assignmentsDataSubmissionsData['metadata_event_time'] <= endDate)]
assignmentsDataSubmissionsData

In [ ]:
assignmentsDataSubmissionsData.drop(['body_submitted_at', 'body_attempt'], axis = 1, inplace = True)
assignmentsDataSubmissionsData

In [ ]:
assignmentInfoData = pd.read_csv(str(input()))
assignmentInfoData

In [ ]:
#assignmentInfoData.drop(['metadata_event_time', 'body_context_id', 'body_due_at'], axis = 1, inplace = True)
#assignmentInfoData

In [ ]:
assignmentsDataSubmissionsData = assignmentsDataSubmissionsData.merge(assignmentInfoData, on = ['metadata_context_id', 'body_assignment_id'])
assignmentsDataSubmissionsData

In [ ]:
courseStudentsFilesPath = str(input())

In [ ]:
courseStudentsFiles = glob.glob(courseStudentsFilesPath + dirGen.getDelimiter() + '**' + dirGen.getDelimiter() + '*.csv', recursive=True)

In [ ]:
courseStudentsData = pd.DataFrame()
for courseStudentsFile in courseStudentsFiles:
    courseStudentsData = courseStudentsData.append(pd.read_csv(courseStudentsFile), ignore_index = True)
    
courseStudentsData

In [ ]:
courseStudentsData['metadata_event_time'] = courseStudentsData['metadata_event_time'].apply(parseTimestamp)
courseStudentsData.dropna(subset=['metadata_event_time'], inplace = True)
courseStudentsData = courseStudentsData.loc[(courseStudentsData['metadata_event_time'] >= startDate) & (courseStudentsData['metadata_event_time'] <= endDate)]
courseStudentsData

In [ ]:
courseStudentsData = courseStudentsData.groupby(['metadata_context_id', 'metadata_user_id'])['metadata_event_time'].count().to_frame('total').reset_index()
courseStudentsData.drop(['total'], axis = 1, inplace = True)
courseStudentsData

In [ ]:
courseStudentsData = courseStudentsData.loc[courseStudentsData['metadata_context_id'].isin(assignmentInfoData['metadata_context_id'].unique())]
courseStudentsData

In [ ]:
courseStudentAssignmentsComplete = pd.DataFrame()

for course in assignmentInfoData['metadata_context_id'].unique():
    courseStudentAssignmentsCompleteCourse = pd.DataFrame({'body_assignment_id': assignmentInfoData.loc[assignmentInfoData['metadata_context_id'] == course]['body_assignment_id'].unique()}).merge(pd.DataFrame({'metadata_user_id': courseStudentsData.loc[courseStudentsData['metadata_context_id'] == course]['metadata_user_id'].unique()}), how = 'cross')
    courseStudentAssignmentsCompleteCourse['metadata_context_id'] = course
    courseStudentAssignmentsComplete = courseStudentAssignmentsComplete.append(courseStudentAssignmentsCompleteCourse, ignore_index = True)

courseStudentAssignmentsComplete

In [ ]:
assignmentsDataCommentsData = assignmentsDataCommentsData.merge(assignmentsDataSubmissionsData[['body_submission_id', 'body_assignment_id', 'metadata_context_id', 'body_user_id']].rename(columns={'body_user_id': 'metadata_user_id'}), on=['body_submission_id', 'metadata_context_id', 'metadata_user_id'])
assignmentsDataCommentsData

In [ ]:
assignmentsDataCommentsData.drop(['body_submission_id', 'submission_comments_count'], axis = 1, inplace = True)
assignmentsDataCommentsData

In [ ]:
assignmentsDataCommentsData = assignmentsDataCommentsData.sort_values(by='metadata_event_time').drop_duplicates(subset=['body_assignment_id', 'metadata_context_id', 'metadata_user_id'])
assignmentsDataCommentsData

In [ ]:
assignmentsDataCommentsDataFirstComment = assignmentsDataCommentsData.groupby(['metadata_context_id', 'body_assignment_id'])['metadata_event_time'].min().to_frame('first_submission_time').reset_index()
assignmentsDataCommentsDataFirstComment

In [ ]:
assignmentsDataCommentsData = assignmentsDataCommentsData.merge(assignmentsDataCommentsDataFirstComment, on = ['metadata_context_id', 'body_assignment_id'])
assignmentsDataCommentsData

In [ ]:
def calculateDelay(df):
    return (time.mktime(df['metadata_event_time'].timetuple()) / 3600) - (time.mktime(df['first_submission_time'].timetuple()) / 3600)

assignmentsDataCommentsData['delay_comments'] = assignmentsDataCommentsData.apply(calculateDelay, axis = 1)
assignmentsDataCommentsData.drop(['metadata_event_time', 'first_submission_time'], axis = 1, inplace = True)
assignmentsDataCommentsData

In [ ]:
assignmentsDataCommentsData = courseStudentAssignmentsComplete.merge(assignmentsDataCommentsData, how='left', on=['body_assignment_id', 'metadata_context_id', 'metadata_user_id'])
assignmentsDataCommentsData

In [ ]:
assignmentsDataCommentsData = assignmentsDataCommentsData.fillna(((time.mktime((endDate + timedelta(seconds = 1)).timetuple()) / 3600) - (time.mktime(startDate.timetuple()) / 3600)))
assignmentsDataCommentsData

In [ ]:
assignmentsDataCommentsData = assignmentsDataCommentsData.groupby(['metadata_context_id', 'metadata_user_id'])['delay_comments'].mean().to_frame('ave_delay').reset_index()
assignmentsDataCommentsData

In [ ]:
assignmentsDataCommentsData.rename(columns={"ave_delay": "ave_delay_C"}).to_csv("AssignmentCommentsDelay_Intercession_2022.csv", header = True, index = False)